# Programming and Database Fundamentals for Data Scientists - EAS503

## A practical introduction to IPython Notebooks
## Some notes about installation:

### Don't do this:

```
sudo apt-get install ipython-notebook
```

### Instead, do this:

```
pip install ipython tornado pyzmq
```


Otherwise, follow the instructions at: [https://docs.continuum.io/anaconda/install/](https://docs.continuum.io/anaconda/install/)

You can start IPython notebook by running

```
jupyter notebook
```


## Setting up mysql database
In this course we will use mysql database. You can install it following instructions here:
[https://dev.mysql.com/downloads/mysql/](https://dev.mysql.com/downloads/mysql/)

After this step, you should have a mysql database running on your laptop.
### Installing python bindings for mysql
To be able to connect to the mysql database, you need to install the interface (MySQLdb) that will let you connect to the database from within a python application. More information here:

[http://mysql-python.sourceforge.net/](http://mysql-python.sourceforge.net/)

In Unix-like environments (including MacOS), you can try:
```
pip install mysql-python
```


### Demonstrating a simple data science pipeline.
Data available from [Chicago Crime Data](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2)

Before starting to play with the Chicago data, you will need to import the data into your database. We will assume that the database server is running on localhost on the standard port (see mysql help for handling other settings), the username is `username` and the password is `password` (You should use better usernames and passwords!)

#### Step 1: Getting the data
Download the data from [here](https://www.cse.buffalo.edu/ubds/docs/chicago_crime_data.csv). This is only data for 2015 onwards. You can also download the full dataset here as well [here](https://www.cse.buffalo.edu/ubds/docs/chicago_crime_data_all.csv).

#### Step 2: Setting up the database
Create a new database and the empty table using the lines below. You can either go into `mysql` command prompt and copy them, or just copy them into a text file (`script.sql`) and use the command line prompt:

```
mysql -u username -p < script.sql
```

You will be prompted for your password.
```
-- create database
create database if not exists eas503db;
use eas503db;
-- create table
drop table if exists `chicago_crime_data`;
create table `chicago_crime_data` (
  ID mediumint(9),
  Case_Number text,
  Date varchar(32),
  Block text,
  IUCR text,
  Primary_Type text,
  Description text,
  Location_Description text,
  Arrest varchar(6),
  Domestic varchar(6),
  Beat text,
  District text,
  Ward int,
  Community_Area text,
  FBI_Code text,
  X_Coordinate float,
  Y_Coordinate float,
  Year int(4),
  Updated_On varchar(32),
  Latitude float,
  Longitude float,
  Location varchar(64)
);
```

#### Step 3: Importing data into the database.
We will use `mysqlimport` to do the bulk import. You may use the direct INFILE LOAD command from the `mysql` prompt, which does the same thing.

```
mysqlimport --local --fields-terminated-by=, --fields-enclosed-by='"' --ignore-lines=1 -u root -proot eas503db chicago_crime_data.csv
```

#### Step 4: Converting the date elements to the correct data type
You need final few adjustments to make sure that the date fields are stored as the correct `datetime` type.

```
update chicago_crime_data set Date =  STR_TO_DATE(Date, '%m/%d/%Y %h:%i:%s %p');
alter table chicago_crime_data modify Date datetime;
update chicago_crime_data set Updated_On =  STR_TO_DATE(Updated_On, '%m/%d/%Y %h:%i:%s %p');
alter table chicago_crime_data modify Updated_On datetime;
```

In [1]:
import MySQLdb

In [2]:
# Start the connection to database
# make sure to close it after your are done
db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     passwd="root",       # your password
                     db="eas503db")    # name of the data base

### Let us start with a very simple query
Get the total number of rows in the database

In [3]:
querystr = 'SELECT count(*) FROM chicago_crime_data'
cur = db.cursor()
cur.execute(querystr)

1

In [4]:
for row in cur.fetchall():
    print("Total number of cases are:")
    print(row)
    

Total number of cases are:
(6414902,)


### Doing more interesting things with SQL
Which are the more crime ridden areas in Chicago since 2016?

In [6]:
querystr = 'SELECT count(*) as cnt, District FROM chicago_crime_data WHERE Date > str_to_date(\'2016/01/01\',\'%Y/%m/%d\') GROUP BY District ORDER BY cnt'
cur = db.cursor()
cur.execute(querystr)

23

In [7]:
for row in cur.fetchall():
    print(row1)

### Zooming into Jefferson Park (011)

In [8]:
querystr = 'SELECT count(*) as cnt, Primary_Type FROM chicago_crime_data WHERE District = "011" AND Date > str_to_date(\'2016/01/01\',\'%Y/%m/%d\') GROUP BY Primary_Type ORDER BY cnt'
cur = db1.cursor()
cur.execute(querystr)

NameError: name 'db1' is not defined

In [13]:
for row in cur.fetchall():
    print(row)

(1, 'NON-CRIMINAL')
(1, 'NON-CRIMINAL (SUBJECT SPECIFIED)')
(1, 'NON - CRIMINAL')
(2, 'HUMAN TRAFFICKING')
(2, 'PUBLIC INDECENCY')
(3, 'CONCEALED CARRY LICENSE VIOLATION')
(4, 'OBSCENITY')
(11, 'INTIMIDATION')
(12, 'STALKING')
(18, 'LIQUOR LAW VIOLATION')
(21, 'KIDNAPPING')
(49, 'ARSON')
(73, 'SEX OFFENSE')
(86, 'GAMBLING')
(144, 'HOMICIDE')
(158, 'CRIM SEXUAL ASSAULT')
(203, 'OFFENSE INVOLVING CHILDREN')
(231, 'INTERFERENCE WITH PUBLIC OFFICER')
(274, 'PUBLIC PEACE VIOLATION')
(644, 'CRIMINAL TRESPASS')
(720, 'WEAPONS VIOLATION')
(745, 'PROSTITUTION')
(787, 'BURGLARY')
(847, 'DECEPTIVE PRACTICE')
(1212, 'MOTOR VEHICLE THEFT')
(1571, 'ROBBERY')
(1768, 'OTHER OFFENSE')
(2200, 'ASSAULT')
(2825, 'CRIMINAL DAMAGE')
(3327, 'THEFT')
(5707, 'NARCOTICS')
(6364, 'BATTERY')


In [ ]:
db.close()

### PYTHON Library PANDAS
One can directly read into a pandas data frame from a database

In [8]:
import pandas as pd

In [9]:
df = pd.read_sql('SELECT * FROM chicago_crime_data limit 10', con=db)

In [10]:
df

,ID,Case_Number,Date,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,...,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Updated_On,Latitude,Longitude,Location
0,2130451,HH370782,2002-05-14 11:05:00,042XX N LINCOLN AVE,1330,CRIMINAL TRESPASS,TO LAND,STREET,true,false,...,47,5,26,1161500.0,1928060.0,2002,2016-04-15 08:55:02,41.9583,-87.6816,"(41.958265994, -87.681638768)"
1,2130452,HH371865,2002-05-14 18:30:00,035XX W MONTROSE AVE,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,BOWLING ALLEY,true,false,...,33,14,11,1151800.0,1929010.0,2002,2016-04-15 08:55:02,41.9611,-87.7173,"(41.961078175, -87.717311252)"
2,2130453,HH370756,2002-05-13 14:15:00,114XX S EGGLESTON AVE,0460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",false,false,...,34,49,08B,1175340.0,1829150.0,2002,2016-04-15 08:55:02,41.6866,-87.6337,"(41.686553786, -87.633737393)"
3,2130454,HH368289,2002-05-13 08:00:00,013XX S KEELER AVE,0430,BATTERY,AGGRAVATED: OTHER DANG WEAPON,STREET,false,false,...,24,29,04B,1148600.0,1893540.0,2002,2016-04-15 08:55:02,41.8638,-87.7300,"(41.863818515, -87.729989801)"
4,2130456,HH368415,2002-05-13 10:30:00,075XX S CONSTANCE AVE,0545,ASSAULT,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",true,false,...,8,43,08A,1189840.0,1855360.0,2002,2016-04-15 08:55:02,41.7581,-87.5798,"(41.758138498, -87.579797849)"
5,2202800,HH462192,2002-06-23 18:35:00,028XX W LUIS MUNOZ MARIN DR N,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,PARK PROPERTY,true,false,...,26,24,24,1157080.0,1910260.0,2002,2016-04-15 08:55:02,41.9095,-87.6984,"(41.909510016, -87.698398735)"
6,2130457,HH365745,2002-05-11 23:15:20,043XX S LAMON AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,false,false,...,23,56,14,1144590.0,1875330.0,2002,2016-04-15 08:55:02,41.8139,-87.7452,"(41.813910365, -87.745164685)"
7,2130458,HH368466,2002-05-05 11:00:00,041XX S WELLS ST,0820,THEFT,$500 AND UNDER,STREET,false,false,...,3,37,06,1175310.0,1877300.0,2002,2016-04-15 08:55:02,41.8187,-87.6324,"(41.818689442, -87.632418468)"
8,2130460,HH364031,2002-05-11 06:15:00,005XX E 71ST ST,0610,BURGLARY,FORCIBLE ENTRY,BAR OR TAVERN,false,false,...,6,69,05,1180890.0,1858060.0,2002,2016-04-15 08:55:02,41.7658,-87.6125,"(41.765758187, -87.612541077)"
9,2130461,HH372769,2002-05-14 08:15:00,0000X E MONROE ST,0820,THEFT,$500 AND UNDER,STREET,false,false,...,42,32,06,1177000.0,1899950.0,2002,2016-04-15 08:55:02,41.8808,-87.6255,"(41.880801792, -87.625515824)"


In [ ]:
df = pd.read_sql(querystr, con=db)

In [42]:
def converter():
    celsius = eval(input("What is the Celsius temperature? "))
    f = celsius*(9/5) + 32
    print("Temperature in Fahrenheit is %d"%f)

In [43]:
converter()

What is the Celsius temperature? 23.7
Temperature in Fahrenheit is 74


In [30]:
location= eval(input("What is your current location? "))

What is your current location? int(4)


In [34]:
eval("4")

4

In [12]:
print("Hello World\n")

Hello World



### Some pythonisms

In [13]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


#### simultaneous assignments

In [5]:
x = 5
y = 3
print(x)
print(y)

5
3


In [6]:
x,y = 5,3
print(x)
print(y)

5
3


In [7]:
sm,df = x-y,x+y
print(sm)
print(df)

2
8


In [9]:
# swapping values
x,y = 5,3
y,x = x,y
print(x)
print(y)

3
5


In [14]:
type(x)

int